In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from copy import deepcopy
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from scipy.ndimage import gaussian_filter
from os.path import join as oj
import sys
import acd
from tqdm import tqdm
import pickle as pkl
import torchvision
import models
import time
from functools import partial
sys.path.append('..')
sys.path.append('../..')
# plt.style.use('dark_background')
from visualize import *
import visualize as viz
from transforms_torch import bandpass_filter
from numpy.fft import *
from data import *
from style import *
data_path = './cosmo'
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
from acd_wooseok.acd.scores import cd

# load dataset and model

In [ ]:
# params
img_size = 256
class_num = 1

In [ ]:
# cosmo dataset
transformer = transforms.Compose([ToTensor()])
mnu_dataset = MassMapsDataset(oj(data_path, 'cosmological_parameters.txt'),  
                              oj(data_path, 'z1_256'),
                              transform=transformer)

# dataloader
data_loader = torch.utils.data.DataLoader(mnu_dataset, batch_size=64, shuffle=False, num_workers=4)

# load model
model = models.load_model(model_name='resnet18', device=device, data_path=data_path)

# model prediction
with torch.no_grad():
    result = {'y': [], 'pred': []}
    for i in tqdm(range(100)):
        sample = mnu_dataset[i]
        x = sample['image']
        result['y'].append(sample['params'][class_num].item())
        result['pred'].append(model(x.unsqueeze(0).to(device)).flatten()[class_num].item())
# print(result)
plt.scatter(result['y'], result['pred'])
plt.xlabel('true param')
plt.ylabel('predicted param')
plt.show()

# conv sparse coding

In [ ]:
# import modules
from funcs import *
from matfac import *

In [ ]:
# set hyper-params and variables
n_components = 20
kernel_size = 7
n_dim = kernel_size + (img_size-1)

csc_dict = Filter(kernel_size, n_components).to(device)
csc_weight = FeatureMap(n_dim, n_components).to(device)
# load model
# csc_dict.load_state_dict(torch.load('./models/csc_dict.pth'))
# csc_weight.load_state_dict(torch.load('./models/csc_weight.pth'))

criterion = nn.MSELoss()
# set optimizer
lamb = 0.05
optimizer_d = Optimizer(params=csc_dict.parameters(), proxs=[prox_normalization], lr=0.005, momentum=0.)
optimizer_w = Optimizer(params=csc_weight.parameters(), proxs=[partial(prox_soft_threshold, lamb=0.005*lamb)], lr=0.005, momentum=0.)

X = iter(data_loader).next()['image'][0:1].to(device)
n_epoch = 10
n_inner = 100
losses_epoch = np.zeros((n_epoch,))
losses_inner = []

lamb_cd = 0.1

In [ ]:
# run prox-SGD
for epoch in range(n_epoch):
    # update dict
    optimizer_d.zero_grad() # need to clear the old gradients
    optimizer_w.zero_grad() # need to clear the old gradients
    X_ = csc_dict(csc_weight.maps)
    loss = torch.norm(X-X_)**2/2
    atoms = get_atoms(csc_dict.convs, csc_weight.maps)
    loss -= lamb_cd * cd.cd(X, model, mask=None, model_type='resnet18', device='cuda', 
                    transform=partial(conv_sparse_coder, atoms=atoms, comp_idx=list(range(5))))[0].flatten()[class_num]
    loss.backward()
    # loss
    losses_epoch[epoch] = losses_epoch[epoch] + loss.data.item() + lamb*L1Norm(csc_weight.parameters())
    # update step
    optimizer_d.step()
    
    # inner loop
    for i in range(n_inner):
        # update weight
        optimizer_d.zero_grad() # need to clear the old gradients
        optimizer_w.zero_grad() # need to clear the old gradients
        X_ = csc_weight(csc_dict.convs)
        loss = torch.norm(X-X_)**2/2
        atoms = get_atoms(csc_dict.convs, csc_weight.maps)
        loss -= lamb_cd * cd.cd(X, model, mask=None, model_type='resnet18', device='cuda', 
                        transform=partial(conv_sparse_coder, atoms=atoms, comp_idx=list(range(5))))[0].flatten()[class_num]
        loss.backward()
        # loss
        losses_inner.append(loss.data.item() + lamb*L1Norm(csc_weight.parameters()))
        # update step
        optimizer_w.step()
        
    if epoch % 2 == 0:
        # recon-error, nnz
        err = torch.norm(X-X_).data.item() / torch.norm(X).data.item() 
        nnz = 0
        for feature_map in csc_weight.maps:
            nnz += np.count_nonzero(feature_map.data.cpu().numpy())
        nnz_W = np.count_nonzero(nnz)/(n_dim*n_dim*n_components)
        
        print('\rTrain Epoch: {} [({:.0f}%)]\tError: {:.6f}\tLoss: {:.6f}\tNNZ: {:.6f}%'.format(
            epoch, 100. * epoch / n_epoch, err, loss.data.item()+lamb*L1Norm(csc_weight.parameters()), 100. * nnz_W), end='')    


In [ ]:
print('recon error:', (torch.norm(X-X_)**2).item()/2)
# with torch.no_grad():
#     print('lamb_cd * cd_score:', lamb_cd * acd.cd(X, model, mask=None, model_type='mnist', device='cuda', 
#                         transform=partial(conv_sparse_coder, atoms=atoms, comp_idx=list(range(5))))[0].flatten()[class_num])
print('lamb * L1Norm:', lamb*L1Norm(csc_weight.parameters()))

In [ ]:
# plot losses
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
# plt.plot(np.log(losses_epoch))
plt.plot((losses_epoch))
plt.title('losses per epoch')

plt.subplot(1,2,2)
# plt.plot(np.log(losses_inner))
plt.plot((losses_inner))
plt.title('losses per inner loop')
plt.show()

In [ ]:
img = X.squeeze().cpu().data.numpy()
imgr = csc_dict(csc_weight.maps).squeeze().cpu().data.numpy()

plt.figure(figsize=(10,10))
plt.subplot(1, 3, 1)
cshow(img)
plt.title('Original')
plt.axis('off')
plt.subplot(1, 3, 2)
cshow(imgr)
plt.title('Reconstructed')
plt.axis('off')
plt.subplot(1, 3, 3)
cshow(img-imgr)
plt.title('Difference')
plt.axis('off')
plt.show()

In [ ]:
from skimage.transform import rescale

n_row = 4
n_col = 5
Nimages = len(csc_dict.convs)

plt.figure(figsize=(15,15))
# plot filters
plt.subplot(1, 2, 1)
p = kernel_size + 2
mosaic = np.zeros((p*n_row,p*n_col))
indx = 0
for i in range(n_row):
    for j in range(n_col):
        im = csc_dict.convs[indx].weight.data.cpu().squeeze().numpy()
        im = (im-np.min(im))
        im = im/np.max(im)
        mosaic[i*p:(i+1)*p,j*p:(j+1)*p] = np.pad(im,(1,1),mode='constant')
        indx += 1

plt.imshow(rescale(mosaic,4,mode='constant'), cmap='magma')
plt.axis('off')    

# plot each atom 
plt.subplot(1, 2, 2)
p = img_size + 2
mosaic = np.zeros((p*n_row,p*n_col))
indx = 0
for i in range(n_row):
    for j in range(n_col):
        im = csc_dict.convs[indx](csc_weight.maps[indx]).data.cpu().squeeze().numpy()
        mosaic[i*p:(i+1)*p,j*p:(j+1)*p] = np.pad(im,(1,1),mode='constant',constant_values=0.)
        indx += 1

plt.imshow(rescale(mosaic,4,mode='constant'), cmap='magma')
plt.axis('off')
plt.show()        

# CD score

In [ ]:
# compute cd-scores
scores = []
atoms = get_atoms(csc_dict.convs, csc_weight.maps)
for comp_idx in range(n_components):
    with torch.no_grad():
        score = cd.cd(X, model, mask=None, model_type='resnet18', device='cuda', 
                transform=partial(conv_sparse_coder, atoms=atoms, comp_idx=[comp_idx]))[0].flatten()[class_num].item()
        scores.append(score)

In [ ]:
activations = []
for param in csc_weight.parameters():
    activations.append(torch.abs(param.data).sum())

In [ ]:
def get_residual(im: torch.Tensor, atoms: list):
    recon = 0
    for atom in atoms:
        recon += atom
    return im - recon

In [ ]:
plt.plot(np.arange(n_components), 50*np.array(scores), label='cd score')
plt.plot(np.arange(n_components), activations, label='activations')
plt.legend()
plt.show()
with torch.no_grad():
    score = cd.cd(X, model, mask=None, model_type='resnet18', device='cuda', 
        transform=partial(get_residual, atoms=atoms))[0].flatten()[class_num].item()
print('cd score for the residual:', 50*score)

In [ ]:
with torch.no_grad():
    score = cd.cd(X, model, mask=None, model_type='resnet18', device='cuda', 
        transform=partial(get_residual, atoms=atoms))[0].flatten()[class_num].item()
print(score)